In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql3

sns.set(style="darkgrid")
ROOT_PATH="C:/Users/moshe/Documents/projectML/p1"
DATA_PATH = "%s/data/raw" % ROOT_PATH
DB_FILE = "%s/hairsalon.db" % DATA_PATH
def readSQL(query):
    import pandas as pd
    import sqlite3 as sql3
    db = sql3.connect(DB_FILE)
    df = pd.read_sql_query(query, db)
    db.close()
    return(df)

db=sql3.connect(DB_FILE)
from sklearn.feature_selection import SelectFromModel

In [25]:
query="""select * from finalbooking"""
final=readSQL(query)
for c in final.columns:
    if final[c].nunique() < 14:
        final[c] = final[c].astype('category')
X=final.loc[:,final.columns[1:]]
y=final['noshowflag']

## Univariate Analysis

In [26]:
import pyMechkar as mechkar
x=final.columns.tolist()
tab1 = mechkar.pyMechkar().Table1(x=x,y="noshowflag",data=final, maxcat=5,miss=3, catmiss=True)

Factorizing... please wait
*********************************************************************************************************************************************************************************
[]
[**********************************************************************************************************************************************************************************
[**********************************************************************************************************************************************************************************
[]
------ Finished in 117.65895318984985econds -----


In [27]:
tab1[tab1['p_value']<0.05]

,Variables,Categories,Population,Category_1,Category_0,p_value
8,previousnoshow,1,174.00 ( 9.70%),24.00 ( 17.40%),150.00 ( 9.10%),0.002
13,svcavg,Mean (SD),65.40 ( 36.70),50.50 ( 53.20),66.70 ( 34.70),0.000
20,x0_wkday_1.0_hour_10.0,1,56.00 ( 3.10%),11.00 ( 8.00%),45.00 ( 2.70%),0.002
114,x0_wkday_4.0_hour_9.0,1,35.00 ( 2.00%),7.00 ( 5.10%),28.00 ( 1.70%),0.015
124,x0_wkday_5.0_hour_14.0,1,48.00 ( 2.70%),0.00 ( 0.00%),48.00 ( 2.90%),0.047
154,x1_MISC,1,68.00 ( 3.80%),31.00 ( 22.50%),37.00 ( 2.20%),0.000
156,x1_STYLE,0,"1,423.00 ( 79.40%)",82.00 ( 59.40%),"1,341.00 ( 81.00%)",0.000
164,x2_CBAL,1,20.00 ( 1.10%),7.00 ( 5.10%),13.00 ( 0.80%),0.000
188,x2_CON,1,56.00 ( 3.10%),30.00 ( 21.70%),26.00 ( 1.60%),0.000
194,x2_FRI,1,4.00 ( 0.20%),4.00 ( 2.90%),0.00 ( 0.00%),0.000


In [28]:
vn1 = tab1.loc[tab1['p_value']<0.05,'Variables'].unique()
print(len(vn1))
vn1

16


array(['previousnoshow', 'svcavg', 'x0_wkday_1.0_hour_10.0',
       'x0_wkday_4.0_hour_9.0', 'x0_wkday_5.0_hour_14.0', 'x1_MISC',
       'x1_STYLE', 'x2_CBAL', 'x2_CON', 'x2_FRI', 'x2_SHCW',
       'x6_daysnoshow8-15', 'x9_10.0', 'x10_6.0', 'x11_2.0', 'x11_3.0'],
      dtype=object)

In [29]:
varList = pd.DataFrame({'Variable': final.columns[1:]})
print(len(varList))
varList['Univariate'] = 0
varList.loc[varList['Variable'].isin(vn1), 'Univariate'] = 1
varList.head()

177


,Variable,Univariate
0,firstapp,0
1,existingcustomer,0
2,multipleappt,0
3,previousnoshow,1
4,last90dproductbuyer,0


### Variable Selection using LASSO (L1 penalization)

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
lassomod = LogisticRegression(penalty='l1',random_state=0).fit(X,y)
model = SelectFromModel(lassomod, prefit=True)
varList['Lasso'] = model.get_support().astype('int64')

C:\Users\moshe\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Variable Selection using Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
rfc=RandomForestClassifier(random_state=0,n_estimators=100,max_depth=2)
rfmod = RandomForestClassifier().fit(X, y)
model = SelectFromModel(rfmod, prefit=True)
varList['RandomForest'] = model.get_support().astype('int64')

C:\Users\moshe\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### Variable Selection using gradient boosting

In [32]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel

gbmod = GradientBoostingClassifier().fit(X, y)
model = SelectFromModel(gbmod, prefit=True)
varList['GradientBoost'] = model.get_support().astype('int64')


### Variable Selection using LinearSVC

In [33]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

svmmod = LinearSVC(C=0.01, penalty="l1",dual=False,random_state=0).fit(X, y)
model = SelectFromModel(svmmod, prefit=True)
varList['LinearSVC'] = model.get_support().astype('int64')

### Variable Selection using XGB

In [34]:
import xgboost as xgb
classifier=xgb.XGBClassifier()
finalnum=final.copy()
finalnum.dtypes
columnstoconvert=finalnum.loc[:,finalnum.dtypes=='category'].columns
finalnum[columnstoconvert]=finalnum[columnstoconvert].astype(bool)
Xnum=finalnum.loc[:,finalnum.columns[1:]]
ynum=finalnum['noshowflag']
finalnum.dtypes
xgbmod=classifier.fit(Xnum,ynum)

In [35]:
from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(xgbmod, prefit=True)
varList['xgb'] = model.get_support().astype('int64')

### Variable Selection using AdaBoost

In [36]:
from sklearn.ensemble import AdaBoostClassifier
adamod=AdaBoostClassifier().fit(X,y)
model = SelectFromModel(adamod, prefit=True)
varList['AdaBoost'] = model.get_support().astype('int64')


### Variable Selection using Decision Tree

In [37]:
from sklearn.tree import DecisionTreeClassifier
treemod=DecisionTreeClassifier().fit(X,y)
model = SelectFromModel(treemod, prefit=True)
varList['DecTree'] = model.get_support().astype('int64')


In [38]:
if 'Sum' in varList.columns:
    varList=varList.drop(columns=['Sum'])
varList['Sum'] =  np.sum(varList,axis=1)

In [39]:
selectedfeatures=varList.loc[varList['Sum']>=4,'Variable']
selectedfeatures

0                    firstapp
2                multipleappt
3              previousnoshow
6                      svcavg
7                  lastsvcavg
8      x0_wkday_1.0_hour_10.0
9      x0_wkday_1.0_hour_11.0
18      x0_wkday_1.0_hour_8.0
36     x0_wkday_3.0_hour_14.0
44     x0_wkday_4.0_hour_10.0
55      x0_wkday_4.0_hour_9.0
60     x0_wkday_5.0_hour_14.0
74                   x1_COLOR
76                   x1_STYLE
80                    x2_CBAL
83                    x2_CDPB
92                     x2_CON
95                     x2_FRI
97                     x2_SBD
102                   x2_SHCW
106                  x3_BECKY
110                  x3_KELLY
111                 x3_SINEAD
115                     x4_JJ
122          x5_lastappt31-45
133    x7_last60dlatecancel-1
136      x8_last60regcancel-0
140                   x9_10.0
142                   x9_12.0
147                   x9_17.0
154                   x10_5.0
155                   x10_6.0
157                   x11_2.0
158       

In [41]:
filepath = "C:/Users/moshe/Documents/projectML/p1/data/interim/varList.xlsx"
varList.to_excel(filepath)
filepath = "C:/Users/moshe/Documents/projectML/p1/data/interim/selectedfeatures.csv"
selectedfeatures.to_csv(filepath,index=False,header=True)